In [ ]:
%pip install labtech

In [ ]:
import labtech as lt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
@lt.task
class BowTask:

    def run(self):
        txt_train = fetch_20newsgroups(subset='train')
        txt_test = fetch_20newsgroups(subset='test')
    
        vectorizer = CountVectorizer(binary=True)
        bow_train = vectorizer.fit_transform(txt_train.data)
        bow_test = vectorizer.transform(txt_test.data)
        return (bow_train, bow_test, txt_train.target, txt_test.target)

@lt.task
class ClassificationTask:
    data_task: BowTask
    leaf_max: int

    def run(self):
        bow_train, bow_test, target_train, target_test = self.data_task.result
        classifier = RandomForestClassifier(max_leaf_nodes=self.leaf_max, random_state=1)
        classifier.fit(bow_train, target_train)
        target_pred = classifier.predict(bow_test)
        return accuracy_score(target_test, target_pred)

In [ ]:
experiments = [
    ClassificationTask(
        data_task=BowTask(),
        leaf_max=leaf_max,
    )
    for leaf_max in [10, 50, 90]
]
lab = lt.Lab(storage=None)
results = lab.run_tasks(experiments)
results